In [ ]:
def missing(df) : 
    missing_number = df.isnull().sum().sort_values(ascending = False)
    missing_percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending = False)
    missing_values = pd.concat([missing_number, missing_percent], axis = 1, keys = ['Missing_number', 'Missing_percent'])
    return missing_values 

def categorize(df) :
    Quantitive_features = df.select_dtypes([np.number]).columns.tolist()
    Categorical_features = df.select_dtypes(exclude = [np.number]).columns.tolist()
    Discrete_features = [col for col in Quantitive_features if len(df[col].unique()) < 200]
    Continuous_features = [col for col in Quantitive_features if col not in Discrete_features]
    print("Quantitive feautres : {} \nDiscrete features : {} \nContinous features : {} \nCategorical features : {}\n"
     .format(Quantitive_features, Discrete_features, Continuous_features, Categorical_features))
    print("Number of quantitive feautres : {} \nNumber of discrete features : {} \nNumber of continous features : {} \nNumber of categorical features : {}"
     .format(len(Quantitive_features), len(Discrete_features), len(Continuous_features), len(Categorical_features)))
    return Quantitive_features, Categorical_features, Discrete_features, Continuous_features
    
def unique(df) : 
    tb1 = pd.DataFrame({'Columns' : df.columns, 'Number_of_Unique' : df.nunique().values.tolist(),
                       'Sample1' : df.sample(1).values.tolist()[0], 'Sample2' : df.sample(1).values.tolist()[0], 
                       'Sample3' : df.sample(1).values.tolist()[0],
                       'Sample4' : df.sample(1).values.tolist()[0], 'Sample5' : df.sample(1).values.tolist()[0]})
    return tb1
    
def data_glimpse(df) :  
      # Dataset preview 
    print("1. Dataset Preview \n")
    display(df.head())
    print("-------------------------------------------------------------------------------\n")
    
    # Columns imformation
    print("2. Column Imformation \n")
    print("Dataset have {} rows and {} columns".format(df.shape[0], df.shape[1]))
    print("\n") 
    print("Dataset Column name : {}".format(df.columns.values))
    print("\n")
    categorize(df)
    print("-------------------------------------------------------------------------------\n")
    
    # Basic imformation table 
    print("3. Missing data table : \n")
    display(missing(df))
    print("-------------------------------------------------------------------------------\n")
    
    print("4. Number of unique value by column : \n")
    display(unique(df))
    print("-------------------------------------------------------------------------------\n")
    
    print("5. Describe table : \n")
    display(df.describe())
    print("-------------------------------------------------------------------------------\n")
    
    print(df.info())
    print("-------------------------------------------------------------------------------\n")

In [ ]:
# Data Analysis
import warnings 
warnings.filterwarnings('ignore')
    
import pandas as pd
import numpy as np
import os 
import missingno as msno
import matplotlib.pyplot as plt
    
# Data View
pd.options.display.max_columns = 200

# Import Basic Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
    


In [ ]:
df_raw= pd.read_csv("C:\\Users\\jason\\OneDrive\\Desktop\\NCU\\8525_Multivariate Analysis\\Week 8\\Airport_Quarterly_Passenger_Survey.csv")

In [ ]:
data_glimpse(df_raw)

In [ ]:
#Check Data Type
df_raw.dtypes

In [ ]:
df_raw.head() #preview first five rows

In [ ]:
df_raw.describe() #get summary statistics about the dataset

In [ ]:
df_raw.info() #get additional detail from data i.e., count of non-null

In [ ]:
#Removing any row with NA's in the response variable

df_dropna = df_raw.dropna(subset=['Overall satisfaction'])
df_dropna.describe()

In [ ]:
# Impute median value to NAs
df_impute = df_dropna.fillna(df_dropna.median())
df_impute.describe()

In [ ]:
# Glimpse new data after imputation

data_glimpse(df_impute)

In [ ]:
# Check the average of values for the explanatary variables. Grouped by the response variable.
df_impute.groupby('Overall satisfaction').mean()

In [ ]:
#Removing any row with 0's in the response variable

df_clean = df_impute[(df_impute[['Overall satisfaction']] != 0).all(axis=1)]
df_clean.describe()

In [ ]:
#Correlation matrix

plt.figure(figsize=(20,10))
c= df_clean.corr()
sns.heatmap(c)

In [ ]:
# Create Correlation Matrix
corrmatrix = df_clean.corr()
print(corrmatrix)

In [ ]:
# Bar chart of frequency in the response variable
# series of counts
OScount = df_clean['Overall satisfaction'].value_counts()
# get bar chart
OScount.plot(kind='bar')


In [ ]:
list(df_clean.columns[3:36].values.tolist())

In [ ]:
#Subset of the data
!pip install factor_analyzer  
from factor_analyzer import FactorAnalyzer

x =df_clean[df_clean.columns[3:36]]

fa = FactorAnalyzer()
fa.fit(x, 10)

#Get Eigen values and plot
ev, v = fa.get_eigenvalues()
ev
plt.plot(range(1,x.shape[1]+1),ev)

In [ ]:
#Conduct Shapiro-Wilks normality test 
from scipy import stats

for i in df_clean.columns[3:36]:
    print([i])
    a,b = stats.shapiro(df_clean[[i]])
    print("Statistics", a, "p-value", b)
    if b < a:
        print("The null hypothesis can be rejected")
    else:
        print("the null hypothesis cannot be rejected")

In [ ]:
!pip install scipy
from scipy.stats import studentized_range


In [ ]:
!pip install pingouin
from pingouin import multivariate_normality

In [ ]:
#Perform Mardia multivariate normality test
multivariate_normality(x,alpha =.05)

In [ ]:
#Kaiser-Meyer-Olkin (KMO) Measure of Samnpling Adequacy (MSA)

from factor_analyzer.factor_analyzer import calculate_kmo

kmo_all,kmo_model=calculate_kmo(x)
print(kmo_model)



In [ ]:
#Bartlet's test of sphericity
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity

chi_square_value,p_value=calculate_bartlett_sphericity(x)
print(chi_square_value, p_value)

In [ ]:
#get load from factors
fa = FactorAnalyzer(6, rotation='varimax', method='principal')
fa.fit(x)
loads = fa.loadings_
print(loads)

In [ ]:
#Get factor correlation

fa_corr = fa.corr_
print(fa_corr)

In [ ]:
#Get rotation matrix

fa_rotation = fa.rotation_matrix_
print(fa_rotation)

In [ ]:
#Get factor communalities

fa_commu = fa.get_communalities()
print(fa_commu)

In [ ]:
#Get eigenvalues
fa_ev = fa.get_eigenvalues()
print(fa_ev)

In [ ]:
#Get uniqueness
fa_uniq = fa.get_uniquenesses()
print(fa_uniq)

In [ ]:
#Create Factors

factor1 = df_clean[['Wait time at passport inspection', 'Courtesy of inspection staff', 'Courtesy of security staff', 'Thoroughness of security inspection', 'Wait time of security inspection', 'Feeling of safety and security' ]]
factor2 = df_clean[['Ease of making connections', 'Courtesy of airport staff', 'Availability of banks/ATM/money changing', 'Shopping facilities', 'Shopping facilities (value for money)', 'Internet access' ]]
factor3 = df_clean[['Ease of finding your way through the airport', 'Flight information screens','Walking distance inside terminal' ]]
factor4 = df_clean[['Ground transportation to/from airport', 'Parking facilities', 'Parking facilities (value for money)', 'Availability of baggage carts']]
factor5 = df_clean[['Efficiency of check-in staff', 'Check-in wait time', 'Courtesy of of check-in staff']]
factor6 = df_clean[['Restaurants','Restaurants (value for money)']]

In [ ]:
#Get cronback alpha

import pingouin as pg

f1_alpha = pg.cronbach_alpha(factor1)
f2_alpha = pg.cronbach_alpha(factor2)
f3_alpha = pg.cronbach_alpha(factor3)
f4_alpha = pg.cronbach_alpha(factor4)
f5_alpha = pg.cronbach_alpha(factor5)
f6_alpha = pg.cronbach_alpha(factor6)

print(f1_alpha, f2_alpha, f3_alpha, f4_alpha, f5_alpha, f6_alpha, )